# Data Science Capstone Project Intro - the city of Bangkok, Thailand (Cont'd)

After data was collected and clean in the last section, I was summarize these data to easy-to-use format. Then create a new dataset by excel format as below.

### 1. Dataset to Dataframe

In [1]:
import pandas as pd

filename = "bangkok_info.xlsx"
headers = ["Code","District_Eng","District_Thai","Latitude","Longitude", "Population","Residence",
         "Area_Size_sqkm","AVG_Land_Price_sqwa"]

df = pd.read_excel(filename, names = headers)
df.head()

,Code,District_Eng,District_Thai,Latitude,Longitude,Population,Residence,Area_Size_sqkm,AVG_Land_Price_sqwa
0,50,Bang Bon,บางบอน,13.657732,100.360839,106919,50239,34.745,87500
1,6,Bang Kapi,บางกะปิ,13.765833,100.647778,146841,104234,28.523,125000
2,40,Bang Khae,บางแค,13.696111,100.409444,193315,91765,44.456,111250
3,5,Bang Khen,บางเขน,13.873889,100.596389,191323,109112,42.123,86750
4,31,Bang Kho Laem,บางคอแหลม,13.693333,100.502500,88288,39353,10.921,500000


Let's import useful libraries.

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### 2. Analysis data for understand the present situations and find some insight in Bangkok

Base on my research, my domain knowledge and collected dataset, I have found that Bangkok's neighborhood didn't verify clearly.  There is no clear information about neighborhood detail from the government or private organizations.  
Thus, take a look on Foursquare's Place API.

#### First, get Bangkok's latitude and longitude by Nominatim Geolocator

In [7]:
address = 'Bangkok'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("Bangkok: ", latitude, longitude)

Bangkok:  13.7538929 100.8160803


After I have check the data, this latitude and longitude is not at the center of Bangkok Area.
Our ploblem is to find the good area to do food business. 
Thus, I have to use the latitude and longitude of neighborhood "SIAM" ("สยาม" in Thai) where is the hub of shopping center and transportation.
Center area is "BTS Siam Station", find the latitude and longitude data by Google Map Search.


In [20]:
# Siam, Center of Bangkok
latitude = 13.7456058
longitude = 100.5341187
print("Bangkok: ", latitude, longitude)

Bangkok:  13.7456058 100.5341187


## Foursquare API

In [21]:
# Foursquare API request parameters
CLIENT_ID = 'BC1RNYQMUN1EES4UOQSGOF31K00IQMRZRZEETKVR3O5ORI1L' # your Foursquare ID
CLIENT_SECRET = 'DC1BNIOP5GVIYKP5IXHCSKBYDJGNLCPTN0T3YGF02VO153G0' # your Foursquare Secret
VERSION = '20180604'
radius = 50000
LIMIT = 100

# create URL for request explore parameter.
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
print('\nURL:\n' + url)

Your credentails:
CLIENT_ID: BC1RNYQMUN1EES4UOQSGOF31K00IQMRZRZEETKVR3O5ORI1L
CLIENT_SECRET:DC1BNIOP5GVIYKP5IXHCSKBYDJGNLCPTN0T3YGF02VO153G0

URL:
https://api.foursquare.com/v2/venues/explore?client_id=BC1RNYQMUN1EES4UOQSGOF31K00IQMRZRZEETKVR3O5ORI1L&client_secret=DC1BNIOP5GVIYKP5IXHCSKBYDJGNLCPTN0T3YGF02VO153G0&ll=13.7456058,100.5341187&v=20180604&radius=50000&limit=100


Request to find the results.

In [22]:
import requests

results = requests.get(url).json()
'There are {} places around Bangkok.'.format(len(results['response']['groups'][0]['items']))

'There are 100 places around Bangkok.'

Check results' json file.

In [23]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4b377134f964a520154125e3',
  'name': 'Books Kinokuniya (คิโนะคูนิยะ)',
  'location': {'address': 'Siam Paragon',
   'crossStreet': '3rd Fl, #309-314',
   'lat': 13.745838803758621,
   'lng': 100.5349506607895,
   'labeledLatLngs': [{'label': 'display',
     'lat': 13.745838803758621,
     'lng': 100.5349506607895}],
   'distance': 93,
   'postalCode': '10330',
   'cc': 'TH',
   'city': 'ปทุมวัน',
   'state': 'กรุงเทพมหานคร',
   'country': 'ประเทศไทย',
   'formattedAddress': ['Siam Paragon (3rd Fl, #309-314)',
    'ปทุมวัน',
    'กรุงเทพมหานคร 10330',
    'ประเทศไทย']},
  'categories': [{'id': '4bf58dd8d48988d114951735',
    'name': 'Bookstore',
    'pluralName': 'Bookstores',
    'shortName': 'Bookstore',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/bookstore_',
     'suffix': '.png'},
    'primary': Tr

### Define information of interest

In [26]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


### Normalize json to clean dataframe format.

In [27]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Books Kinokuniya (คิโนะคูนิยะ),Bookstore,Siam Paragon,TH,ปทุมวัน,ประเทศไทย,"3rd Fl, #309-314",93,"[Siam Paragon (3rd Fl, #309-314), ปทุมวัน, กรุ...","[{'label': 'display', 'lat': 13.74583880375862...",13.745839,100.534951,NaN,10330,กรุงเทพมหานคร,4b377134f964a520154125e3
1,Tonkatsu Wako (หมูทอด วาโก),Japanese Restaurant,Siam Center,TH,ปทุมวัน,ประเทศไทย,2nd Fl,252,"[Siam Center (2nd Fl), ปทุมวัน, กรุงเทพมหานคร ...","[{'label': 'display', 'lat': 13.74650859238807...",13.746509,100.531974,NaN,10330,กรุงเทพมหานคร,58edbb2214f8f4278e385b8f
2,Siam Paragon (สยามพารากอน),Shopping Mall,991 Rama I Rd,TH,ปทุมวัน,ประเทศไทย,NaN,175,"[991 Rama I Rd, ปทุมวัน, กรุงเทพมหานคร 10330, ...","[{'label': 'display', 'lat': 13.74697922931531...",13.746979,100.534912,ปทุมวัน,10330,กรุงเทพมหานคร,4b0587fdf964a52034ab22e3
3,Gourmet Market (กูร์เมต์ มาร์เก็ต),Supermarket,Siam Paragon,TH,ปทุมวัน,ประเทศไทย,G Fl,105,"[Siam Paragon (G Fl), ปทุมวัน, กรุงเทพมหานคร 1...",NaN,13.746291,100.534789,Pathum Wan,10330,กรุงเทพมหานคร,4b41cdfbf964a52009c925e3
4,centralwOrld (เซ็นทรัลเวิลด์),Shopping Mall,999/9 Rama I Rd,TH,ปทุมวัน,ประเทศไทย,Ratchadamri Rd,601,"[999/9 Rama I Rd (Ratchadamri Rd), ปทุมวัน, กร...",NaN,13.746650,100.539579,NaN,10330,กรุงเทพมหานคร,4b0587fcf964a52002ab22e3
5,Siam Kempinski Hotel Bangkok,Hotel,991/9 Rama I Rd,TH,ปทุมวัน,ประเทศไทย,NaN,313,"[991/9 Rama I Rd, ปทุมวัน, กรุงเทพมหานคร 10330...","[{'label': 'display', 'lat': 13.74826016742340...",13.748260,100.535074,NaN,10330,กรุงเทพมหานคร,4c20b858d8b5b7132385dbec
6,After You (อาฟเตอร์ ยู),Dessert Shop,Siam Paragon,TH,ปทุมวัน,ประเทศไทย,G Fl.,128,"[Siam Paragon (G Fl.), ปทุมวัน, กรุงเทพมหานคร ...","[{'label': 'display', 'lat': 13.74676055646774...",13.746761,100.534202,NaN,10330,กรุงเทพมหานคร,4d1c70a47e10a35d342eff82
7,RBSC Fitness Club,Gym / Fitness Center,Royal Bangkok Sports Club,TH,ปทุมวัน,ประเทศไทย,Henri Dunant Rd,693,"[Royal Bangkok Sports Club (Henri Dunant Rd), ...","[{'label': 'display', 'lat': 13.73967638922438...",13.739676,100.536097,NaN,10330,กรุงเทพมหานคร,4bfde0aef7c82d7fa0e98e04
8,Siam Center (สยามเซ็นเตอร์),Shopping Mall,430 Rama I Rd,TH,ปทุมวัน,ประเทศไทย,NaN,117,"[430 Rama I Rd, ปทุมวัน, กรุงเทพมหานคร 10330, ...",NaN,13.746179,100.533201,NaN,10330,กรุงเทพมหานคร,4b8a56c2f964a520226932e3
9,Paragon Cineplex (พารากอน ซีนีเพล็กซ์),Multiplex,Siam Paragon,TH,ปทุมวัน,ประเทศไทย,5th-6th Fl,106,"[Siam Paragon (5th-6th Fl), ปทุมวัน, กรุงเทพมห...","[{'label': 'display', 'lat': 13.74587902661586...",13.745879,100.535062,NaN,10330,กรุงเทพมหานคร,4b0d131ff964a520a04323e3


**Eventhough Foursquare's databases, "neighborhood" of Bangkok is almost "NaN" that is very difficult to match.
Thus, I will use "District" (or "city" column in Foursquare) to analysis instead.**

### Now, let's visualize the population in Bangkok by district.

In [43]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Siam',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

## Test Map

In [135]:
#sample df

d = {'District': ['Bang Na', 'Wattana'], 'Count': [100, 50]}
bangkokdf = pd.DataFrame(data=d)
bangkokdf


,District,Count
0,Bang Na,100
1,Wattana,50


In [141]:
# world_geo = r'testtt.json' # geojson file

# # create a plain world map
# world_map = folium.Map(location=[latitude, longitude], zoom_start=10, tiles='openstreetmap')

In [165]:
world_geo = r'bangkok_geo.json' # geojson file

# create a plain world map
world_map = folium.Map(location=[latitude, longitude], zoom_start=11, tiles='openstreetmap')

# generate choropleth map using the total immigration of each country to Canada from 1980 to 2013
world_map.choropleth(
    geo_data=world_geo,
    data=bangkokdf,
    columns=['District', 'Count'],
    key_on='feature.properties.NAME_2',
    fill_color='YlOrRd', 
    fill_opacity=0.5, 
    line_opacity=0.9,
    legend_name='Thailand'
)

# add Ecco as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Siam',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(world_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(world_map)

# display map
world_map